In [ ]:
# ACF

In [ ]:
match.arg
match.call
parent.frame
quote 
stats::pacf


In [ ]:

def acf (x, lag_max = None, type = ["correlation", "covariance", "partial"], plot = True, na_action = na_fail, demean = True):

    type = match.arg(type)

    if (type == "partial"):
        m = match.call()
        m[[1L]] = quote(stats::pacf)
        m$type = NULL
        return(eval(m, parent.frame()))
    
    series = deparse1(substitute(x))
    x = na_action(as_ts(x))
    x.freq = frequency(x)
    x = as_matrix(x)

    if (not is.numeric(x)):
        stop("'x' must be numeric")
    sampleT = as.integer(nrow(x))
    nser = as.integer(ncol(x))

    if (is_na(sampleT) or is_na(nser)):
        stop("'sampleT' and 'nser' must be integer")

    if (is_null(lag_max)):
        lag_max = floor(10 * (log10(sampleT) - log10(nser)))

    lag_max = int(min(lag_max, sampleT - 1L))

    if (is_na(lag_max) or lag_max < 0):
        stop("'lag_max' must be at least 0")

    if (demean): 
        x = sweep(x, 2, colMeans(x, na_rm = TRUE), check_margin = FALSE)

    lag = matrix(1, nser, nser)
    lag[lower.tri(lag)] = -1
    acf = .Call(C_acf, x, lag_max, type == "correlation")
    lag = outer(0:lag_max, lag/x.freq)
    acf.out = structure(list(acf = acf, type = type, n_used = sampleT, 
        lag = lag, series = series, snames = colnames(x)), _class = "acf")

    if (plot):
        plot.acf(acf.out, ...)
        invisible(acf.out)

    else acf.out